# Simple
One Application with a dependency to one Library

In [1]:
from cicd_sim import *

In [2]:
# have reproducible build artifact names
import random
random.seed(123)

## Create Infrastructure

In [3]:
output = StdOutput()
bb = Repos()
conan = Conan(output)
artifactory = Artifactory()
build_id = BuildIdGenerator()
jenkins = Jenkins(artifactory, conan, bb, output, build_id)

## Initiate Library

In [4]:
lib = bb.create_repo('lib', jenkins)
lib_dev = lib.create_branch('develop', 'blue')
lib_dev.commit_file('VERSION', '1.0.0')
lib_dev.push()

BUILDING            lib[develop]
        PUBLISH     lib[develop]               + lib/1.0.0-200206113900+0000003


## Initiate Application

In [5]:
app = bb.create_repo('app', jenkins)
app_dev = app.create_branch('develop', 'magenta')
app_dev.commit_file('VERSION', '1.0.0')
app_dev.commit_file('REQUIRES', 'lib/>1.0.0-0')
app_dev.push()

BUILDING            app[develop]
    INSTALLING      app[develop]               * lib/1.0.0-200206113900+0000003
        PUBLISH     app[develop]               + app/1.0.0-200209125600+0000004


## Create Library 'Feature A'

In [6]:
feature_a = lib_dev.create_branch('feature/A', 'light_gray')
feature_a.commit_file('Feature A File', 'AAAAA')
feature_a.push()

BUILDING            lib[feature/A]
        PUBLISH     lib[feature/A]             + lib/1.0.0-200210125600+0000005
BUILDING            app[develop]
    INSTALLING      app[develop]               * lib/1.0.0-200210125600+0000005
        PUBLISH     app[develop]               + app/1.0.0-200215042000+0000004


# Merge feature to 'develop'

In [7]:
lib_dev.merge(feature_a)
lib_dev.push()

BUILDING            lib[develop]
        PUBLISH     lib[develop]               + lib/1.0.0-200218051900+0000006
BUILDING            app[develop]
    INSTALLING      app[develop]               * lib/1.0.0-200218051900+0000006
        PUBLISH     app[develop]               + app/1.0.0-200219105500+0000004


# Trigger a manual 'Application' build

In [8]:
jenkins.build(app_dev)

BUILDING            app[develop]
    INSTALLING      app[develop]               * lib/1.0.0-200218051900+0000006
        PUBLISH     app[develop]               + app/1.0.0-200219213200+0000004


# Trigger a manual 'Library' build
This will cause yet another application build because the version of the library increased

In [9]:
jenkins.build(lib_dev)

BUILDING            lib[develop]
        PUBLISH     lib[develop]               + lib/1.0.0-200224051500+0000006
BUILDING            app[develop]
    INSTALLING      app[develop]               * lib/1.0.0-200224051500+0000006
        PUBLISH     app[develop]               + app/1.0.0-200301075200+0000004


# Create 'Library release' branch

In [10]:
lib_r125 = lib_dev.create_branch('release/R125', 'yellow')
lib_r125.push()

BUILDING            lib[release/R125]
        PUBLISH     lib[release/R125]          + lib/1.0.0-rc.200307173700+0000007
BUILDING            app[develop]
    INSTALLING      app[develop]               * lib/1.0.0-rc.200307173700+0000007
        PUBLISH     app[develop]               + app/1.0.0-200311123100+0000004


# Bump library version

In [11]:
lib_dev.commit_file('VERSION', '1.1.0').push()

BUILDING            lib[develop]
        PUBLISH     lib[develop]               + lib/1.1.0-200315094600+0000008
BUILDING            app[develop]
    INSTALLING      app[develop]               * lib/1.1.0-200315094600+0000008
        PUBLISH     app[develop]               + app/1.0.0-200316000800+0000004


# Create 'Application release' branch

In [12]:
app_release = app_dev.create_branch('release/R125', 'yellow').commit_file('REQUIRES', 'lib/>1.0.0-0 <=1.0.0')
app_release.push()
# app_dev.push()
# app_dev.get_file_content('REQUIRES')

BUILDING            app[release/R125]
    INSTALLING      app[release/R125]          * lib/1.0.0-rc.200307173700+0000007
        PUBLISH     app[release/R125]          + app/1.0.0-rc.200317195500+0000006


# Fix an issue for R125

In [13]:
lib_r125.commit_file('FixR125', 'A pretty cool fix').push()

BUILDING            lib[release/R125]
        PUBLISH     lib[release/R125]          + lib/1.0.0-rc.200319085900+0000009
BUILDING            app[release/R125]
    INSTALLING      app[release/R125]          * lib/1.0.0-rc.200319085900+0000009
        PUBLISH     app[release/R125]          + app/1.0.0-rc.200323051500+0000006


# Merge R125 fix back to develop

In [14]:
lib_dev.merge(lib_r125).push()

BUILDING            lib[develop]
        PUBLISH     lib[develop]               + lib/1.0.0-200329143700+0000010


# Finalize R125

In [15]:
lib_master = lib.checkout('master')
lib_master.merge(lib_r125)
lib_r125.delete()
lib_master.push()

BUILDING            lib[master]
        PUBLISH     lib[master]                + lib/1.0.0
BUILDING            app[release/R125]
    INSTALLING      app[release/R125]          * lib/1.0.0
        PUBLISH     app[release/R125]          + app/1.0.0-rc.200405050500+0000006
